Import required packages

In [ ]:
import os
import json
import csv
import xml.etree.ElementTree as ET
import urllib
import urllib.request
import shutil
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import http.cookiejar
from bs4 import BeautifulSoup
!pip install elsapy
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
!pip install scidownl
!pip install slate3k
import slate3k as slate
from scidownl.scihub import *
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

The task was to get the urls of the articles which have an open access.

Steps followed:

Go to the following link:

https://www.sciencedirect.com/search/advanced?qs=highlights&articleTypes=FLA&accessTypes=openaccess&show=100&offset=3600 


*   This link is obtained by filtering out the documents with "open access" and keyword "highlights"

*   Click on Export and select Export citations to text

*   We get a list of urls







In [ ]:
def extractURLs(fileContent):
     urls = re.findall('\(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', fileContent.lower())

    #  for url in urls:
    #         print(url)
     return urls

Getting the urls from the citation files and storing it in variable

In [ ]:
myFile = open(os.getcwd()+"path"+ "file_name",encoding="utf8")
fileContent = myFile.read()
listURL=(extractURLs(fileContent))

Pass the API key generated from Elsevier portal

In [ ]:
key = 'API KEY'
client = ElsClient(key)

We pass the extracted URLs which are stored in the list and download the scientific articles in .xml format

In [ ]:
listPII=[]
for url in listURL:
    if url.startswith("(http://www.sciencedirect.com"):
        pii = url.replace("(http://www.sciencedirect.com/science/article/pii/", "").replace(")", "").replace("\n", "")
        if not pii in listPII:
            listPII.append(pii)

Downloading the files in XML format






In [ ]:
for pii in listPII:
    piir = "http://api.elsevier.com/content/article/PII:" + pii + "?httpAccept=text/xml&APIKey=" + key
    local_filename, headers = urllib.request.urlretrieve(piir)
    shutil.copy(local_filename, "folder_path/" +pii + ".xml")

Method to split the text into sentences

In [ ]:
def getSentences(text):
  sentences=[]
  intersent=''
  for s in re.split('[.][\s]([A-Z])',text):
    if len(s)==1:
      intersent=s
    else:
      if len(re.split('[\s]',intersent+s))>4:  
        if not '\n' in  intersent+s:     
          sentences.append(intersent+s)
        intersent=''
  return sentences

Method to get the "Abstract" section from the papers

In [ ]:
def getAbstract():
  abstract_df = {}
  abstract_dict={}
  for filename in os.listdir(directory):
    abstract = []
    sentence_index = []
    if filename.endswith("xml") and filename in finalvalid:
      tree = ET.parse(directory + "/" + filename)
      root = tree.getroot()
      abstract_text = ""
      for tag in tree.iter():
        if(tag.tag == '{http://purl.org/dc/elements/1.1/}description'):
            abstract_text+= str(tag.text)  
      
      abstract_df[filename]=getSentences(abstract_text) 
      abstract_dict[filename]=abstract_text                                            
  return abstract_df,abstract_dict

Method to get the "Highlights" section from the papers


In [ ]:
def getHighlights():
    highlight_df={}
    for filename in os.listdir(directory):
        all_files = []
        if filename.endswith(".xml"):
            isIntroFound = False
            all_files.append(filename)
            tree = ET.parse(directory + "/" + filename)
            highlightswithoutpreprocess = ""
            for tag in tree.iter():

                if (tag.tag == '{http://www.elsevier.com/xml/common/dtd}abstract-sec'):
                    for t in tag:
                        if (t.tag == '{http://www.elsevier.com/xml/common/dtd}simple-para'):

                            for tg in t:
                                if (tg.tag == '{http://www.elsevier.com/xml/common/dtd}list'):

                                    for g in tg:
                                        if (g.tag == '{http://www.elsevier.com/xml/common/dtd}list-item'):

                                            for ti in g:
                                                if (ti.tag == '{http://www.elsevier.com/xml/common/dtd}para'):

                                                    highlightswithoutpreprocess += str(ti.text)
                                                    highlightswithoutpreprocess += " "
            if(highlightswithoutpreprocess!=""):
                highlight_df[filename]=highlightswithoutpreprocess
    return highlight_df

Method to get the "Introduction" section from the papers

In [ ]:
def getIntro():
    intro_df = {}
    intro_dict={}
    for filename in os.listdir(directory):
        introduction = []
        sentence_index = []
        if filename.endswith(".xml") and filename in finalvalid:
            isIntroFound = False
            tree = ET.parse(directory + "/" + filename)
            root = tree.getroot()
            introText=""
            for child in root.iter():
                if child.tag == '{http://www.elsevier.com/xml/common/dtd}section' and isIntroFound == False:
                    for introtag in child:
                      if introtag.tag=='{http://www.elsevier.com/xml/common/dtd}section-title' and introtag.text=='Introduction':
                        isIntroFound=True
                      if introtag.tag=='{http://www.elsevier.com/xml/common/dtd}para' or introtag.tag == '{http://www.elsevier.com/xml/common/dtd}section':
                        introText+=" "+introtag.text
                        for subintroTag in introtag:
                            introText+=" "+str(subintroTag.text) 
                            introText+=" "+subintroTag.tail       
            intro_df[filename]=getSentences(introText)
            intro_dict[filename]=introText
    return intro_df, intro_dict

Method to get the "All other" section from the paper

In [ ]:
def getOtherContent():
  other_content_df = {}
  othercontent_dict={}
  for filename in os.listdir(directory):
    oc = []
    sentence_index = []
    if filename.endswith("xml") and filename in finalvalid:
      tree = ET.parse(directory + "/" + filename)
      root = tree.getroot()
      oc_text = ""
      for child in root.iter():
        if child.tag == '{http://www.elsevier.com/xml/common/dtd}section':
            for introtag in child:
              if introtag.tag=='{http://www.elsevier.com/xml/common/dtd}para':
                oc_text+=" "+introtag.text
                for subintroTag in introtag:
                    oc_text+=" "+str(subintroTag.text) 
                    oc_text+=" "+subintroTag.tail   
      other_content_df[filename] = getSentences(oc_text)
      othercontent_dict[filename]= oc_text
  return other_content_df, othercontent_dict

Extract keywords, font specific words and title words

In [ ]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))
def getkeywords():
  bold_words = {}
  italic_words = {}
  keyword_main = {}
  title_words = {}
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~\n'''
  all_files = []
  for filename in os.listdir(directory):
      title = []
      bold = []
      italics = []
      keywords = []
      if filename.endswith(".xml"):
          all_files.append(filename)
          tree = ET.parse(directory + "/" + filename)
          for tag in tree.iter():
              if (tag.tag == '{http://www.elsevier.com/xml/common/dtd}bold'): #get bold words
                  bold.append(tag.text)

                  bold_words[filename] = (bold)

          for tag in tree.iter():
              if (tag.tag == '{http://www.elsevier.com/xml/common/dtd}italic'): #get italic words
                  italics.append(tag.text)
                  italic_words[filename] = (italics)

          for tag in tree.iter():
              if (tag.tag == '{http://www.elsevier.com/xml/common/dtd}keyword'):  # get keywords
                  for t in tag:
                      if (t.tag == '{http://www.elsevier.com/xml/common/dtd}text'):
                          keywords.append(t.text)
                          

          for tag in tree.iter():
              if(tag.tag == '{http://purl.org/dc/elements/1.1/}title'): # title
                for word in (tag.text).split(" "):
                  if word not in punctuations and word not in stop_words and len(word)>3:
                    keywords.append(word)
          keyword_main[filename] = (keywords)
  return bold_words, italic_words, keyword_main

Method for tokenisation, removing stopwords and lemmatization

In [ ]:
def getValidTokens(sent):
  stopWords = set(stopwords.words('english'))
  tokenizer = RegexpTokenizer(r'\w+')
  words = tokenizer.tokenize(sent)
  wordsFiltered = []

  for w in words:
      w = str(w).lower()
      if w not in stopWords:
          wordsFiltered.append((w))
  wordnet_lemmatizer = WordNetLemmatizer()
  lemmatized_words = [wordnet_lemmatizer.lemmatize(token, pos="v") for token in wordsFiltered]
  return lemmatized_words

Downloading the papers as the pdf files and separting the content of paper into "Abstract", "Highlights", "Introduction" and "All other" section


In [ ]:
def getAbstractHighlightsKeywords(pii_id):
  keywordsres=[]
  xml_url='https://www.sciencedirect.com/science/article/abs/pii/'+pii_id


  site= "http://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/getHistoricalData.jsp?symbol=JPASSOCIAT&fromDate=1-JAN-2012&toDate=1-AUG-2012&datePeriod=unselected&hiddDwnld=true"
  hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}

  req = urllib.request.Request(xml_url, headers=hdr)

  try:
      page = urllib.request.urlopen(req)
  except (urllib.HTTPError, e):
      print (e.fp.read())

  content = page.read()
  soup = BeautifulSoup(content, 'html.parser')
  keywords=soup.findAll("div", {"class": "keyword"})
  for keyword in keywords:
    keywordsres.append(keyword.get_text())
  results = soup.find(id='abstracts')
  
  strR=''
  for res in results:
    strR+=(res.getText())
  
  abstract=strR.split('Abstract')[1]
  Highlights=strR.split('Abstract')[0]
  return abstract,Highlights.split('Highlights')[1],keywordsres

In [ ]:
Abstract={}
Highlights={}
OtherContent={}
OtherContent_Dict={}
Abstract_Dict={}
Highlights_Dict={}
Intro={}
Intro_Dict={}
keywordmain={}
for pii_id in listPII:
  pii_doc =FullDoc(sd_pii=pii_id)
  docid=''
  if pii_doc.read(client):
      print ("pii_doc.title: ", pii_doc.id)
      docid=pii_doc.id
      pii_doc.write()
      abstract,highlight,keywords=getAbstractHighlightsKeywords(pii_id)
      out = 'paper'
      try:
        sci = SciHub(docid[4:len(docid)], out).download(choose_scihub_url_index=1)
        paperName=(pii_doc.title[0:len(pii_doc.title)-1])
        punc=['/','?',':']
        for p in punc:
          if p in paperName:
            paperName=paperName.replace(p," ")
        paperName=paperName.replace("'","’")
      
        with open('paper/'+paperName+'.pdf','rb') as f:
          extracted_text = slate.PDF(f)
          text = [blk.replace("\n", " ") for blk in extracted_text]
          resText="\r\n".join(text)
        if len(resText.split('Introduction'))==2:
          contText=(resText.split('Introduction')[1]).split('References')[0]
        else:
          contText=''
          for t in range(len(resText.split('Introduction'))):
            if t!=0:
              contText+=resText.split('Introduction')[t]
        intro=getIntroDictCont(contText)
        if intro=='':
          raise Exception
        else:
          Intro[pii_id]=getSentences(intro)
          Intro_Dict[pii_id]=intro

        OtherContent[pii_id]=getSentences(contText)
        OtherContent_Dict[pii_id]=contText
        
        Abstract_Dict[pii_id]=(abstract)
        Highlights_Dict[pii_id]=(highlight)
        Abstract[pii_id]=getSentences(abstract)
        Highlights[pii_id]=getSentences(highlight)
        keywordmain[pii_id]=keywords
      except:
        print("Failed to read:"+pii_doc.title[0:len(pii_doc.title)-1]+"with pii_id="+pii_id)


  else:
      print ("Read document failed.")